# Reproducibility

## Paper

### “Drosophila anti-nematode and antibacterial immune regulators revealed by RNA-Seq,” BMC Genomics 
### https://doi.org/10.1186/s12864-015-1690-2

In [ ]:
%%bash
# get public key and log on to AWS

##Get Data

#RNA-seq Data
#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?token-szetgcuwtvsvhyf&acc=GSE61466

# Sample: XGWIE_Symbiotic30h
fastq-dump -I --split-files -O datasets --gzip SRR1574318

# Sample: XGWIE_Uninfectedcontrol
fastq-dump -I --split-files -O datasets --gzip SRR1573961

# Reference Genome
#ftp://ftp.ensembl.org/pub/release-78/fasta/drosophila_melanogaster/dna/
 #wget -r --no-parent ftp://ftp.ensembl.org/pub/release-78/fasta/drosophila_melanogaster/dna/
wget ftp://ftp.ensembl.org/pub/release-78/fasta/drosophila_melanogaster/dna/Drosophila_melanogaster.BDGP5.dna.toplevel.fa.gz
    
# .gtf file
wget ftp://ftp.ensembl.org/pub/release-78/gtf/drosophila_melanogaster/Drosophila_melanogaster.BDGP5.78.gtf.gz

In [ ]:
%%bash
##Get Software
# Bioconda
# Steps to set up bioconda
# 1. Download miniconda
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh

# 2. log out the server
# 3. set up channels
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge

## to install packages
conda install -y r

## to search for packages
conda search bwa

## Reference: https://bioconda.github.io

# Install SRAtoolkit, HTseq, R, DEseq, cufflinks from Bioconda
conda install -y sra-tools htseq r deseq cufflinks

#FastQC
conda install -y fastqc #(Do we need fastqc??)

#Bowtie
wget https://sourceforge.net/projects/bowtie2/2.3.4.3/bowtie2-2.3.4.3-linux-x86_64.zip
unzip bowtie2-2.3.4.3-linux-x86_64.zip
sudo mv bowtie2-2.3.4.3-linux-x86_64 /usr/local/bin
PATH=$PATH:/usr/local/bin/bowtie2-2.3.4.3-linux-x86_64
export PATH

rm bowtie2-2.3.4.3-linux-x86_64.zip

#Tophat
wget http://ccb.jhu.edu/software/tophat/downloads/tophat-2.1.1.Linux_x86_64.tar.gz
tar xvfz tophat-2.0.0.Linux_x86_64.tar.gz
sudo mv tophat-2.0.0.Linux_x86_64 /usr/local/bin
PATH=$PATH:/usr/local/bin/tophat-2.0.0.Linux_x86_64
export PATH

rm tophat-2.0.0.Linux_x86_64.tar.gz
# Sam tools



## Alignment reads and coverage analysis
**requires that gtf file be unzipped and tophat, bowtie, and samtools be installed**
***

### Generate bowtie index files for Drosophila genome 

In [ ]:
%%bash

bowtie2-build --large-index \
~/datasets/Drosophila/Drosophila_melanogaster.BDGP5.dna.toplevel.fa.gz \
~/datasets/Drosophila_melanogaster.BDGP5.dna.toplevel 

### Perform alignment using genome index

In [ ]:
%%bash
  
tophat -r 101 \
       --segment-mismatches 2 \
       --segment-length 30 \
       -x 25 \
       -G ~/datasets/Drosophila/Drosophila_melanogaster.BDGP5.78.gtf \
       -p 2 ~/datasets/Drosophila_melanogaster.BDGP5.dna.toplevel \
       ~/datasets/SRR1573961_1.fastq.gz ~/datasets/SRR1573961_2.fastq.gz

In [ ]:
#Differential gene expression analysis

#Calculate RPKM
#R Version 3.3.1

#Generate data frame with Data
Gene_Name = c("A","B",'C','D')
Length_in_KB = c(2,4,1,10)
Rep1_Counts = c(10, 20, 5, 0)
Rep2_Counts = c(12, 25, 8, 0)
Rep3_Counts = c(30, 60, 15, 1)
df = data.frame(Gene_Name,Length_in_KB,Rep1_Counts,Rep2_Counts,Rep3_Counts)

#Find RPM and RPKM
scalingFactor = colSums(df[3:5])/10^6 
RPM = cbind(df[,1:2],t(t(df[3:5])/scalingFactor))
RPKM = cbind(RPM[1:2],RPM[,3:5]/RPM[,2])

#based on https://www.rna-seqblog.com/rpkm-fpkm-and-tpm-clearly-explained/


In [ ]:
#Transcript analysis using CUFFLINKS


%%bash
cufflinks -g ~/datasets/Drosophila/Drosophila_melanogaster.BDGP5.78.gtf -b ~/datasets/Drosophila_melanogaster.BDGP5.dna.toplevel.fa ~/tophat_SRR1574318/accepted_hits.bam

In [ ]:
#Differential transcript analysis using Cuffdiff

In [ ]:
#Gene ontology analysis (GO)

#Website
#"https://david.ncifcrf.gov/summary.jsp"

In [ ]:
#Differential gene expression analysis using DESeq and General Linear Model


%%bash
# read count generated using HTSeq
htseq-count --stranded=no -f SRR157accepted_hits.bam T47D_accepted_hits.bam merged.gtf > 


In [ ]:
%%R
require(DESeq)
require(Biobase)

# import the read counts for DESeq
countTable <- read.table('result', header=FALSE, row.names=1)
colnames(countTable) <- c('Ax_12h', 'Ax_30h', 'Sym_12h', 'Sym_30h', 'Ph_12h', 'Ph_30h', 'Ctrl')
condition <- c('Ax', 'Ax', 'Sym', 'Sym', 'Ph', 'Ph', 'Ctrl')
condition <- factor(condition)

# get the countDataSet instance
cds <- newCountDataSet(countTable, condition)

# normalization
cds <- estimateSizeFactors(cds)

# variance estimation without replicates
cds <- estimateDispersions(cds, method='blind', sharingMode='fit-only')

# differential expression tests
Ax_Vs_Sym <- nbinomTest(cds, 'Ax', 'Sym')
Sym_Vs_Ph <- nbinomTest(cds, 'Sym', 'Ph')
Ax_Vs_Ph <- nbinomTest(cds, 'Ax', 'Ph')

# GLM tests
# model A
countTable_A <- read.table('result', header=FALSE, row.names=1)[:,c(1:6)]
colnames(countTable_A) <- c('Ax_12h', 'Ax_30h', 'Sym_12h', 'Sym_30h', 'Ph_12h', 'Ph_30h')
condition <- c('Ax', 'Ax', 'Sym', 'Sym', 'Ph', 'Ph')
condition <- factor(condition)

fit1 <- fitNbinomGLMs(cds, count ~ condition)
fit0 <- fitNbinomGLMs(cds, count ~ 1)
pvalsGLM_A <- nbinomGLMTest(fit1, fit0)
padjGLM_A = p.adjust(pvalsGLM, method='BH')

# model B
countTable_B <- read.table('result', header=FALSE, row.names=1)[:,c(1,2,5,6)]
colnames(countTable_B) <- c('Ax_12h', 'Ax_30h', 'Ph_12h', 'Ph_30h')
condition <- c('Ax', 'Ax', 'Ph', 'Ph')
condition <- factor(condition)

fit1 <- fitNbinomGLMs(cds, count ~ condition)
fit0 <- fitNbinomGLMs(cds, count ~ 1)
pvalsGLM_B <- nbinomGLMTest(fit1, fit0)
padjGLM_B = p.adjust(pvalsGLM, method='BH')



####Docker instructions -- Thanks to Steve Tsang
####To run the Docker image 

In [ ]:
%%bash
docker run -it -v `pwd`:`pwd` -w `pwd` flytools /bin/bash